In [1]:
import pandas as pd
import statsmodels.multivariate as sm_m
import statsmodels.api as sm
import statsmodels.formula.api as smf 
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import calendar
import re
import numpy as np
import os

%matplotlib inline 

In [2]:
from p_values import Variance_Inflation_Factor
def variance_inflation_factor(x):
    columns = x.columns
    VIF = Variance_Inflation_Factor(x)
    df_vif = pd.DataFrame({"vif": VIF})
    df_vif = df_vif.set_index(columns)
    return df_vif

In [3]:
[x for x in os.listdir() if re.search("oyof_2021",x)]

['oyof_20210105-tv.xlsx', 'oyof_20210111-tv.xlsx']

In [4]:
data1 = pd.read_excel("oyof_20210105-tv.xlsx")

In [5]:
for col in [x for x in data1.columns if re.search("total_likes",x)]:
    data1[col] = data1[col].apply(lambda x:x/1000)

In [6]:
bin_bin = [x for x in data1.columns if re.search("binary",x)]

In [9]:
numword = [x for x in data1.columns if re.search("num",x)]

# for DHC mobile pc

In [296]:
data_reg = pd.get_dummies(data1, columns=["Week","ori_pdcode_DHC",], drop_first=True)
y=data_reg["sales_qt"]
x=data_reg.drop(columns=["DHC","sales_qt",
                             "ori_pdcode_month","ori_pdcode_week",
                            "ori_pdcode_DHC_week", "ori_pdcode_DHC_month"]+bin_bin)

x = sm.add_constant(x)

In [271]:
data_reg = pd.get_dummies(data1, columns=["Week","ori_pdcode",], drop_first=True)
y=data_reg["sales_qt"]
x=data_reg.drop(columns=["DHC","sales_qt","ori_pdcode_DHC",
                             "ori_pdcode_month","ori_pdcode_week",
                            "ori_pdcode_DHC_week", "ori_pdcode_DHC_month"])

x = sm.add_constant(x)

# for oyof

In [10]:
data_reg = pd.get_dummies(data1, columns=["Week","ori_pdcode",], drop_first=True)
model_32 = ["avg_Physical_Hor_Picture", "avg_Diagonal_dominance_Picture", 
            "avg_Warm Hue_Picture", "avg_faces","DHC_Product_Promo"]
robust_1 = ["avg_Physical_Hor_Picture", "avg_Diagonal_dominance_Picture", 
            "avg_Warm Hue_Picture", "avg_faces","DHC_Product_Promo"]+["avg_Brightness_Picture","avg_Colorfulness_Picture","avg_Contrast_Picture_z","avg_len_text_total"] +numword
robust_2 = ["avg_Physical_Hor_Picture", "avg_Diagonal_dominance_Picture", 
            "avg_Warm Hue_Picture", "avg_faces","DHC_Product_Promo"]+["avg_Brightness_Picture","avg_Colorfulness_Picture","avg_Contrast_Picture_z"]
y=data_reg["sales_qt"]
x=data_reg.drop(columns=["sales_qt","ori_pdcode_month","ori_pdcode_week"
                             ])

x = sm.add_constant(x)

In [ ]:
x.head()

# create model

In [16]:
#ori_pdcode_week for oyof
#ori_pdcode_DHC_week for mobilepc
model= sm.OLS(y, x).fit(cov_type='cluster', cov_kwds={'groups': data_reg["ori_pdcode_week"]})

# Log model

In [86]:
#ori_pdcode_week for oyof
#ori_pdcode_DHC_week for mobilepc
x_log = x.copy()
for col in [x for x in x.columns if re.search("total_likes",x)]:
    x_log[col] = x_log[col].apply(lambda x:np.log1p(x))
model= sm.OLS(y, x_log).fit(cov_type='cluster', cov_kwds={'groups': data_reg["ori_pdcode_week"]})

# Start regression

In [17]:
vif = variance_inflation_factor(x)
df_coeffs = pd.DataFrame({"Coefficients": model.params, "p": model.pvalues, "std-er":model.bse})


In [18]:
#drop(columns = dummy_col)

In [19]:
df_coeffs.index=[a for a in x.columns]
df_coeffs["vif"] = vif 
dummy_col = [x for x in df_coeffs.T.columns if re.search("dummy",x) or (re.search("pdcode",x)) or (re.search("Week_",x))]

In [20]:
tidy_index = list(pd.read_excel("index_dhcmobilepc_totallikes.xlsx").dropna()["var"])
final_idx = tidy_index.copy()
counter = 0
for i,idx in enumerate(tidy_index):
    try:
        #print(i)
        df_coeffs.T.drop(columns = dummy_col).T.loc[idx]
    except:
        print(idx)
        final_idx.pop(i-counter)
        counter = counter+1
        
#len(final_idx)==len(tidy_index)
if (len(list(df_coeffs.T.drop(columns = dummy_col).T.index)) == len(final_idx))==False:
    print (set(list(df_coeffs.T.drop(columns = dummy_col).T.index)).difference(set(list(final_idx))))
add=list(set(list(df_coeffs.T.drop(columns = dummy_col).T.index)).difference(set(list(final_idx))))

other_price_dc
other_sales_yesterday
DHC_Product_Promo
avg_Brightness_Picture
avg_Colorfulness_Picture
avg_Contrast_Picture_z
avg_Contrast_Picture^2
avg_Diagonal_dominance_Picture
avg_Physical_Hor_Picture
avg_Warm Hue_Picture
avg_len_text_total
avg_numPos-Neg
avg_numBrand
avg_numModel
avg_numOYBrand
numProduct_mention


In [91]:
fit_measure = dict()
fit_measure["R-Sq"] = model.rsquared
fit_measure["Adj_R-Sq"] = model.rsquared_adj
fit_measure["AIC"]=model.aic
fit_measure["BIC"]=model.bic
fit_measure["Log-Likehood"] = model.llf
pd.DataFrame(fit_measure, index=["Fit Measure"]).T

,Fit Measure
R-Sq,0.897681
Adj_R-Sq,0.895104
AIC,3868.686617
BIC,4684.530043
Log-Likehood,-1809.343309


In [21]:
np.round_(df_coeffs.T.drop(columns = dummy_col).T.loc[final_idx+add], decimals=8).reset_index()

,index,Coefficients,p,std-er,vif
0,posting_po_binary,-0.012317,6.240948e-01,0.025133,1.869235
1,total_likes_product_only,0.028930,2.207760e-03,0.009452,1.413365
2,posting_pm_binary,-0.005444,8.617601e-01,0.031261,1.361835
3,total_likes_product_model,0.009201,8.026100e-04,0.002745,1.275442
4,posting_mo_binary,0.075609,4.185570e-03,0.026401,7.456476
5,total_likes_by_model_only,-0.023675,1.849124e-01,0.017857,9.609984
6,posting_spillover_binary,0.018300,5.618805e-01,0.031548,2.009513
7,total_likes_total_spillover,0.008382,3.082000e-05,0.002011,1.319297
8,price_dc,-0.151937,0.000000e+00,0.009471,18.313915
9,sales_yesterday,0.000946,1.000000e-08,0.000164,4.340303


In [50]:
styler = df_coeffs.T.drop(columns = dummy_col).T.sort_values(["p"], axis=0, ascending=True).style.bar(subset = ["Coefficients", "p",], align='left', color=['#d65f5f', '#5fba7d'])
#print(len(df_coeffs.T.drop(columns = dummy_col).T))
styler

,Coefficients,p,vif
const,5.394833,0.000000,inf
price_dc,-0.150974,0.000000,18.452486
sales_yesterday,0.000944,0.000000,4.346022
total_likes_total_spillover,0.007941,0.000112,1.326991
total_likes_product_model,0.010181,0.001180,1.315132
total_likes_product_only,0.028308,0.002478,1.416848
oy_sample,0.442836,0.006649,1.083482
oy_display_product,0.090025,0.014835,1.744325
numProduct_mention,-19.409170,0.016055,1.740525
posting_mo_binary,0.056114,0.055286,8.769041


# NBD reg

In [108]:
#ori_pdcode_week for oyof
#ori_pdcode_DHC_week for mobilepc

x_nbd = x.drop(columns=['const'])
y_nbd = np.exp(y)

In [109]:
model= smf.glm(formula = "y_nbd ~ x_nbd", data=data_reg, 
               family=sm.families.NegativeBinomial()).fit(cov_type='cluster', cov_kwds={'groups': data_reg["ori_pdcode_week"]})


In [110]:
fit_measure = dict()
#fit_measure["R-Sq"] = model.rsquared
#fit_measure["Adj_R-Sq"] = model.rsquared_adj
#fit_measure["AIC"]=model.aic
#fit_measure["BIC"]=model.bic
fit_measure["Log-Likehood"] = model.llf
pd.DataFrame(fit_measure, index=["Fit Measure"]).T

,Fit Measure
Log-Likehood,-28794.242604


In [111]:
vif = variance_inflation_factor(x)
df_coeffs = pd.DataFrame({"Coefficients": model.params,,"std-er":model.bse, "p": model.pvalues})

In [112]:
df_coeffs.index

Index(['Intercept', 'x_nbd[0]', 'x_nbd[1]', 'x_nbd[2]', 'x_nbd[3]', 'x_nbd[4]',
       'x_nbd[5]', 'x_nbd[6]', 'x_nbd[7]', 'x_nbd[8]',
       ...
       'x_nbd[114]', 'x_nbd[115]', 'x_nbd[116]', 'x_nbd[117]', 'x_nbd[118]',
       'x_nbd[119]', 'x_nbd[120]', 'x_nbd[121]', 'x_nbd[122]', 'x_nbd[123]'],
      dtype='object', length=125)

In [113]:
df_coeffs.index=["const"]+[a for a in x_nbd.columns]
df_coeffs["vif"] = vif 
dummy_col = [x for x in df_coeffs.T.columns if re.search("dummy",x) or (re.search("pdcode",x)) or (re.search("Week_",x))]

In [114]:
tidy_index = list(pd.read_excel("index_dhcmobilepc_totallikes.xlsx").dropna()["var"])
final_idx = tidy_index.copy()
counter = 0
for i,idx in enumerate(tidy_index):
    try:
        #print(i)
        df_coeffs.T.drop(columns = dummy_col).T.loc[idx]
    except:
        print(idx)
        final_idx.pop(i-counter)
        counter = counter+1
        
#len(final_idx)==len(tidy_index)
if (len(list(df_coeffs.T.drop(columns = dummy_col).T.index)) == len(final_idx))==False:
    print (set(list(df_coeffs.T.drop(columns = dummy_col).T.index)).difference(set(list(final_idx))))
add=list(set(list(df_coeffs.T.drop(columns = dummy_col).T.index)).difference(set(list(final_idx))))

other_price_dc
other_sales_yesterday
DHC_Product_Promo
avg_Contrast_Picture^2
avg_Diagonal_dominance_Picture
avg_Physical_Hor_Picture
avg_Warm Hue_Picture


In [115]:
np.round_(df_coeffs.T.drop(columns = dummy_col).T.loc[final_idx+add], decimals=6).reset_index()

,index,Coefficients,p,std-er,vif
0,posting_po_binary,0.017698,0.502303,0.026381,2.362963
1,total_likes_product_only,0.023412,0.023402,0.010328,1.417050
2,posting_pm_binary,-0.013027,0.663545,0.029946,1.389213
3,total_likes_product_model,0.009698,0.001242,0.003003,1.317181
4,posting_mo_binary,0.047227,0.105779,0.029198,9.190007
5,total_likes_by_model_only,0.006660,0.679162,0.016103,10.326746
6,posting_spillover_binary,0.014431,0.644829,0.031307,2.049364
7,total_likes_total_spillover,0.006455,0.004054,0.002246,1.328426
8,price_dc,-0.143765,0.000000,0.008935,18.457582
9,sales_yesterday,0.001124,0.000000,0.000186,4.360119


In [116]:
styler = df_coeffs.T.drop(columns = dummy_col).T.sort_values(["p"], axis=0, ascending=True).head(n=20).style.bar(subset = ["Coefficients", "p",], align='left', color=['#d65f5f', '#5fba7d'])
#print(len(df_coeffs.T.drop(columns = dummy_col).T))
styler

,Coefficients,p,std-er,vif
const,6.786016,0.000000,0.207157,inf
price_dc,-0.143765,0.000000,0.008935,18.457582
sales_yesterday,0.001124,0.000000,0.000186,4.360119
avg_Brightness_Picture,-0.515805,0.001034,0.157203,6.838701
total_likes_product_model,0.009698,0.001242,0.003003,1.317181
total_likes_total_spillover,0.006455,0.004054,0.002246,1.328426
numProduct_mention,-19.291180,0.012427,7.717158,1.754918
total_likes_product_only,0.023412,0.023402,0.010328,1.417050
oy_display_product,0.071855,0.037638,0.034566,1.746592
oy_sample,0.264282,0.089574,0.155675,1.084151
